# Predicción del Valor de Casas en California
**Autor**: [Jesús Manuel Muñoz Larguero]
**Matricula**: [207054]
**Descripción**: En este proyecto, utilizaremos un conjunto de datos de viviendas en California para construir y entrenar un modelo de regresión lineal que prediga el valor de las casas. Exploraremos los datos, preprocesaremos las variables necesarias, entrenaremos varios modelos y ajustaremos el mejor modelo para obtener las predicciones más precisas.

In [ ]:
import os
import tarfile
import urllib
import pandas as pd
import numpy as np

DOWNLOAD_ROOT ="https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL =  DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# En esta sección, descargamos el conjunto de datos de viviendas de California,
# que contiene información sobre características de las viviendas y su valor medio.
# Este dataset será útil para entrenar y evaluar un modelo de regresión que prediga
# el valor de las casas en función de sus características.

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

In [ ]:
import matplotlib.pyplot as plt

# Comenzamos con una exploración básica de los datos para entender su estructura.
# Contamos los valores faltantes, visualizamos histogramas y exploramos la correlación
# entre las variables y el valor de las viviendas.

housing.info()  # Información general de las columnas
missing_values = housing.isnull().sum()  # Contamos los valores N/A
print(missing_values)

# Visualizamos un histograma para entender las distribuciones
housing.hist(bins=50, figsize=(20, 15))
plt.show()

# Mostramos la correlación de las variables
corr_matrix = housing.corr()
print(corr_matrix["median_house_value"].sort_values(ascending=False))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Creamos un pipeline para automatizar el preprocesamiento de los datos.
# Esto incluye la imputación de valores faltantes con la mediana y la normalización.
# También añadimos algunas nuevas características como la relación de cuartos por hogar.

# Definir una clase para generar nuevas características
class CombinedAttributesAdder:
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, 3] / X[:, 6]
        population_per_household = X[:, 5] / X[:, 6]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, 4] / X[:, 3]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num.columns)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Comenzamos con un modelo de regresión lineal, ya que es fácil de interpretar
# y es un buen punto de partida. Luego, comparamos su rendimiento con modelos
# más complejos como Árboles de Decisión y Random Forest.

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing["median_house_value"])

# Evaluación del modelo con validación cruzada
scores = cross_val_score(lin_reg, housing_prepared, housing["median_house_value"],
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Para encontrar la mejor combinación de hiperparámetros para el modelo Random Forest,
# usamos GridSearch. Esto nos permite ajustar el número de árboles y características
# para mejorar el rendimiento del modelo sin sobreajustarlo.

param_grid = [
    {"n_estimators": [3, 10, 30], "max_features": [2, 4, 6, 8]},
    {"bootstrap": [False], "n_estimators": [3, 10], "max_features": [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)
grid_search.fit(housing_prepared, housing["median_house_value"])

# Mostrar los mejores parámetros encontrados
grid_search.best_params_

In [ ]:
import joblib
joblib.dump(grid_search.best_estimator_, "best_model.joblib")